In [2]:
import pennylane as qml
import torch
from torch.utils.data import DataLoader, TensorDataset
import ray
from ray import tune
from ray.air import session
from sklearn.model_selection import train_test_split
import numpy as np
import yaml
import json
import time
import os
import pandas as pd


# Custom Libraries
from utils.model import Qkernel
from utils.data_generator import DataGenerator
from utils.agent import TrainModel

# Backend Configuration
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

2024-11-15 21:55:27,970	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-15 21:55:28,623	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
# Read Configs
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

data = np.load('checkerboard_dataset.npy', allow_pickle=True).item()
x_train, x_test, y_train, y_test = data['x_train'], data['x_test'], data['y_train'], data['y_test']

training_data = torch.tensor(x_train, dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(x_test, dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(y_train, dtype=torch.int)
testing_labels = torch.tensor(y_test, dtype=torch.int)

kernel = Qkernel(   
                        device = config['qkernel']['device'], 
                        n_qubits = 4, 
                        trainable = True, 
                        input_scaling = True, 
                        data_reuploading = True, 
                        ansatz = 'embedding_paper', 
                        ansatz_layers = 5
                    )
    
agent = TrainModel(
                        kernel=kernel,
                        training_data=training_data,
                        training_labels=training_labels,
                        testing_data=testing_data,
                        testing_labels=testing_labels,
                        optimizer= 'adam',
                        lr= 0.1,
                        epochs = 200,
                        train_method= 'ccka',
                        target_accuracy=0.95,
                        get_alignment_every=10,  
                        validate_every_epoch=None, 
                        base_path='.',
                        lambda_kao=0.01,
                        lambda_co=0.1,
                        clusters=4
                      )

In [11]:
import torch
from torch.utils.data import DataLoader

def calculate_centroids(train_data, train_labels, num_clusters_per_class):
    unique_classes = torch.unique(train_labels)
    num_classes = unique_classes.size(0)
    
    # Initialize tensors for main centroids and cluster centroids
    main_centroids = []
    cluster_centroids = []
    
    for cls in unique_classes:
        # Get data points belonging to the current class
        class_data = train_data[train_labels == cls]
        
        # Compute main centroid as the mean of all features for the class
        main_centroid = class_data.mean(dim=0)
        main_centroids.append(main_centroid)
        
        # K-Means-like cluster initialization for the class
        n_samples = class_data.size(0)
        if n_samples <= num_clusters_per_class:
            # If not enough samples, use all data points as centroids
            cluster_init = class_data
        else:
            # Randomly initialize cluster centroids
            initial_indices = torch.randperm(n_samples)[:num_clusters_per_class]
            cluster_init = class_data[initial_indices]
            
            # Iteratively refine centroids
            for _ in range(5):  # Number of refinement iterations
                distances = torch.cdist(class_data, cluster_init)  # Compute distances
                cluster_assignments = distances.argmin(dim=1)  # Assign to nearest centroid
                
                # Update centroids to be the mean of assigned points
                for i in range(num_clusters_per_class):
                    cluster_points = class_data[cluster_assignments == i]
                    if cluster_points.size(0) > 0:
                        cluster_init[i] = cluster_points.mean(dim=0)
        
        cluster_centroids.append(cluster_init)
    
    # Stack the centroids into tensors
    main_centroids = torch.stack(main_centroids, dim=0)
    cluster_centroids = torch.cat(cluster_centroids, dim=0)
    
    # Ensure centroids are leaf tensors
    main_centroids.requires_grad_(True)
    cluster_centroids.requires_grad_(True)
    
    return main_centroids, cluster_centroids




torch.manual_seed(42)
n_samples, n_features, n_classes, clusters_per_class = 100, 5, 3, 4
train_data = torch.rand(n_samples, n_features)
train_labels = torch.randint(0, n_classes, (n_samples,))
    
main_centroids, cluster_centroids = calculate_centroids(train_data, train_labels, clusters_per_class)
print("Main Centroids:\n", main_centroids)
print("Cluster Centroids:\n", cluster_centroids)


Main Centroids:
 tensor([[0.4915, 0.4302, 0.4707, 0.5329, 0.4510],
        [0.4466, 0.4058, 0.5046, 0.4990, 0.4453],
        [0.5571, 0.5140, 0.5321, 0.4686, 0.4060]], requires_grad=True)
Cluster Centroids:
 tensor([[0.4103, 0.4986, 0.4936, 0.7673, 0.7676],
        [0.2456, 0.2066, 0.2970, 0.3154, 0.4583],
        [0.6191, 0.4295, 0.7957, 0.7367, 0.2454],
        [0.6221, 0.5363, 0.2217, 0.2414, 0.3570],
        [0.5588, 0.4546, 0.7054, 0.3871, 0.6408],
        [0.6754, 0.3836, 0.1603, 0.5244, 0.4380],
        [0.2054, 0.3524, 0.6492, 0.8243, 0.3363],
        [0.2660, 0.3906, 0.1646, 0.2390, 0.1198],
        [0.9082, 0.7023, 0.8198, 0.2520, 0.1499],
        [0.2482, 0.7609, 0.4719, 0.4069, 0.3340],
        [0.5743, 0.3646, 0.5132, 0.8554, 0.4463],
        [0.6616, 0.3239, 0.4460, 0.1415, 0.5916]], requires_grad=True)


In [9]:
import torch.optim as optim

_kernel_optimizer = optim.SGD([
            {'params': main_centroids, 'lr': 0.1},
            {'params': cluster_centroids, 'lr': 0.1},
        ])

In [ ]:

class_centroids = cluster_centroids[0]
main_centroid = main_centroids[0]

x_0 = main_centroid.repeat(4, 1)
x_1 = class_centroids
                
K = kernel(x_0, x_1).to(torch.float32)

AttributeError: 'list' object has no attribute 'repeat'